<a href="https://colab.research.google.com/github/walkerjian/DailyCode/blob/main/Code_Craft_OverloadingTheQuackStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem:
A quack is a data structure combining properties of both stacks and queues. It can be viewed as a list of elements written left to right such that three operations are possible:

push(x): add a new item x to the left end of the list
pop(): remove and return the item on the left end of the list
pull(): remove the item on the right end of the list.
Implement a quack using three stacks and O(1) additional memory, so that the amortized time for any push, pop, or pull operation is O(1).

##Solution:
To implement a quack using three stacks with O(1) amortized time complexity for push, pop, and pull operations, we can use the following strategy:

1. **Stack Setup:** Utilize three stacks:
   - `left`: For handling push and pop operations (push to and pop from the left end).
   - `right`: For handling pull operations (pop from the right end).
   - `buffer`: Used to transfer elements between the left and right stacks to balance them when needed.

2. **Push Operation (`push(x)`):**
   - Push element `x` onto the `left` stack directly. This is a direct operation and thus O(1) in time complexity.

3. **Pop Operation (`pop()`):**
   - If the `left` stack is not empty, pop the top element from the `left` stack.
   - If the `left` stack is empty, transfer half of the elements from the `right` stack to the `left` stack, reversing their order so they can be popped correctly. After transferring, pop the top element of the `left` stack.
   - This operation has an amortized time complexity of O(1) because each element is moved at most twice: once to the `right` stack and possibly back to the `left` stack.

4. **Pull Operation (`pull()`):**
   - If the `right` stack is not empty, pop the top element from the `right` stack.
   - If the `right` stack is empty, transfer half of the elements from the `left` stack to the `right` stack without reversing their order. After transferring, pop the top element of the `right` stack.
   - Like the pop operation, this has an amortized time complexity of O(1).

By using three stacks and careful handling of element transfers, we ensure that elements are moved infrequently enough to maintain an amortized time complexity of O(1) for all operations. Below is a Python implementation of this concept:

This implementation manages element distribution between the `left` and `right` stacks when one is empty, ensuring that all operations can continue in an O(1) amortized manner.

##Implementation:

In [3]:
class Quack:
    def __init__(self):
        self.left = []
        self.right = []
        self.buffer = []

    def push(self, x):
        self.left.append(x)

    def pop(self):
        if not self.left:
            self._move_half(self.right, self.left)
        if self.left:
            return self.left.pop()
        raise IndexError("pop from empty quack")

    def pull(self):
        if not self.right:
            self._move_half(self.left, self.right)
        if self.right:
            return self.right.pop()
        raise IndexError("pull from empty quack")

    def _move_half(self, source, destination):
        size = len(source) // 2
        # Transfer half of the elements to the buffer
        while len(source) > size:
            self.buffer.append(source.pop())

        # Transfer from buffer to destination in reversed order (only for pop)
        while self.buffer:
            destination.append(self.buffer.pop())


##Testing:

In [2]:
quack = Quack()
quack.push(1)
quack.push(2)
quack.push(3)
print(quack.pop())  # Outputs 3
quack.push(4)
print(quack.pull())  # Outputs 1
print(quack.pop())  # Outputs 2
print(quack.pull())  # Outputs 4

3
4
1
2


##Overloading the Quack Stack:
Implementing a data structure like a quack where the type of operation (push, pop, pull) is determined by how the structure is used in expressions (e.g., as a left-value or right-value) is quite innovative but challenging in statically-typed languages. However, this can be approached in more dynamic or flexible languages like Python, albeit with some limitations and not exactly by lvalue/rvalue differentiation since Python doesn't differentiate these directly in the same way as languages like C++.

In Python, we can create a class where methods are invoked based on the context in which an object of the class is used. While we can't directly check if an object is being used as an lvalue or rvalue, we can provide method interfaces that make such usage clear by their semantic context in the code.

One way to achieve a context-driven interface in Python is by overriding the special methods that Python uses for assignment and other operations. For example, we could use:
- Item assignment for pushing: `quack[0] = value`
- Item retrieval for popping or pulling: `value = quack[0]`

Here's how you could implement a quack with such behavior using Python's magic methods:



In this example, pushing is done by assigning to the key `0`, popping by accessing the key `0`, and pulling by accessing the key `-1`. This kind of usage gives a clear, if somewhat unusual, semantic meaning to the operation based on the index used, which simulates differentiating operations by their context in the code. This is as close as we can get to lvalue/rvalue differentiation in Python while still retaining clarity and functionality.

In [ ]:
class Quack:
    def __init__(self):
        self.left = []
        self.right = []

    def __setitem__(self, key, value):
        if key == 0:
            self.left.append(value)  # Push to left
        else:
            raise KeyError("Only key 0 is allowed for pushing")

    def __getitem__(self, key):
        if key == 0:
            return self._pop()
        elif key == -1:
            return self._pull()
        else:
            raise KeyError("Key must be 0 for pop and -1 for pull")

    def _pop(self):
        if not self.left:
            self._transfer(self.right, self.left)
        if self.left:
            return self.left.pop()
        raise IndexError("pop from empty quack")

    def _pull(self):
        if not self.right:
            self._transfer(self.left, self.right)
        if self.right:
            return self.right.pop()
        raise IndexError("pull from empty quack")

    def _transfer(self, source, destination):
        # Move all items from source to destination to reverse them
        while source:
            destination.append(source.pop())

In [5]:
quack = Quack()
quack[0] = 1  # push
quack[0] = 2  # push
quack[0] = 3  # push
print(quack[0])  # pop -> 3
quack[0] = 4  # push
print(quack[-1])  # pull -> 1
print(quack[0])  # pop -> 2
print(quack[-1])  # pull -> 4

3
1
4
2
